In [1]:
import logging
import sys

from pyspark.errors.exceptions.base import AnalysisException
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
def table_exists(spark: SparkSession, database: str, table: str) -> bool:
    try:
        spark.catalog.getTable(f"{database}.{table}")
        return True
    except AnalysisException:
        return False

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
project_prefix = "crypto-cloud-dev-650251698703"
data_lake_bucket_name = "crypto-cloud-dev-650251698703-data-lake-bucket"
data_lake_iceberg_lock_table_name = "crypto_cloud_dev_650251698703_iceberg_lock_table"
data_prefix = project_prefix.replace("-", "_")

landing_date = "2025-09-27"
symbol = "ADAUSDT"

spark = (
    SparkSession.builder
    .appName("Transform Job")
    .master("local[*]")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1,"
        "org.apache.iceberg:iceberg-aws-bundle:1.6.1,"
        "org.apache.hadoop:hadoop-aws:3.3.4"
    )
    .config("spark.sql.catalog.glue_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.glue_catalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config("spark.sql.catalog.glue_catalog.warehouse", f"s3a://{data_lake_bucket_name}/")
    .config("spark.sql.catalog.glue_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.glue_catalog.lock.table", f"{data_lake_iceberg_lock_table_name}")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
    .config("spark.sql.defaultCatalog", "glue_catalog")
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.sql.parquet.enableVectorizedReader", "false")
    .config("spark.sql.columnVector.offheap.enabled", "false")
    .config("spark.memory.offHeap.enabled", "false")
    .config("spark.sql.catalog.glue_catalog.read.parquet.vectorization.enabled", "false")
    .config("spark.driver.memory", "2g")
    .config("spark.driver.extraJavaOptions", "-XX:MaxDirectMemorySize=1g")
    .config("spark.sql.codegen.wholeStage", "false")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/Users/anhtu/.pyenv/versions/3.11.11/envs/crypto-cloud-batch/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/anhtu/.ivy2/cache
The jars for the packages stored in: /Users/anhtu/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ab5439aa-3497-4b6f-9e87-4439fce5a0c4;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 111ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-aws-bundle;1.6.1 from cent

In [5]:
output_path = (
    f"s3a://{data_lake_bucket_name}/landing_zone/spot/daily/aggTrades/{symbol}/{landing_date}"
)
df = spark.read.parquet(output_path)

25/11/05 11:35:49 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
df = (
    df.withColumn("timestamp_date", F.from_unixtime(F.col("timestamp") / 1_000_000))
    .withColumn("timestamp_second", (F.col("timestamp") / 1_000_000).cast("long"))
    .withColumn("group_id", (F.col("timestamp_second") / 900).cast("long"))
    .withColumn("group_date", F.from_unixtime(F.col("group_id") * 900))
    .withColumn("transform_date", F.current_date())
    .withColumn("transform_timestamp", F.current_timestamp())
    .withColumn("landing_date", F.to_date(F.lit(landing_date), "yyyy-MM-dd"))
    .withColumn("symbol", F.lit(symbol))
)

In [7]:
transform_db = f"{data_prefix}_transform_db"
spark.sql(f"""
CREATE DATABASE IF NOT EXISTS {transform_db}
LOCATION 's3a://{data_lake_bucket_name}/transform_zone/'
""")

DataFrame[]

In [8]:
aggtrade_table = "aggtrades"
if table_exists(spark, transform_db, aggtrade_table):
    df.writeTo(f"{transform_db}.{aggtrade_table}").overwritePartitions()
    logger.info(
        f"Table {transform_db}.{aggtrade_table} overwritten for {symbol} on {landing_date}"
    )
else:
    df.writeTo(f"{transform_db}.{aggtrade_table}").tableProperty(
        "format-version", "2"
    ).partitionedBy("symbol", "landing_date").createOrReplace()
    logger.info(f"Table {transform_db}.{aggtrade_table} created for {symbol} on {landing_date}")

INFO:__main__:Table crypto_cloud_dev_650251698703_transform_db.aggtrades created for ADAUSDT on 2025-09-27


In [9]:
serving_db = f"{data_prefix}_serving_db"
spark.sql(f"""
CREATE DATABASE IF NOT EXISTS {serving_db}
LOCATION 's3a://{data_lake_bucket_name}/serving_zone/'
""")

DataFrame[]

In [10]:
sql_stmt = f"""
select 
    group_id,
    group_date,
    first(timestamp, true) as open_time,
    round(first(price, true), 4) as open_price,
    round(max(price), 4) as high_price,
    round(min(price), 4) as low_price,
    round(last(price, true), 4) as close_price,
    round(sum(quantity), 1) as volume,
    last(timestamp, true) as close_time,
    landing_date,
    symbol
from {transform_db}.{aggtrade_table}
where landing_date = DATE('{landing_date}') AND symbol = '{symbol}'
group by group_id, group_date, landing_date, symbol
"""
logger.info(f"SQL Statement:\n{sql_stmt}")
df_kline = spark.sql(sql_stmt)

INFO:__main__:SQL Statement:

select 
    group_id,
    group_date,
    first(timestamp, true) as open_time,
    round(first(price, true), 4) as open_price,
    round(max(price), 4) as high_price,
    round(min(price), 4) as low_price,
    round(last(price, true), 4) as close_price,
    round(sum(quantity), 1) as volume,
    last(timestamp, true) as close_time,
    landing_date,
    symbol
from crypto_cloud_dev_650251698703_transform_db.aggtrades
where landing_date = DATE('2025-09-27') AND symbol = 'ADAUSDT'
group by group_id, group_date, landing_date, symbol



In [11]:
klines_table = "klines"
if table_exists(spark, serving_db, klines_table):
    df_kline.writeTo(f"{serving_db}.{klines_table}").overwritePartitions()
    logger.info(f"Table {serving_db}.{klines_table} overwritten for {symbol} on {landing_date}")
else:
    df_kline.writeTo(f"{serving_db}.{klines_table}").tableProperty(
        "format-version", "2"
    ).partitionedBy("symbol", "landing_date").createOrReplace()
    logger.info(f"Table {serving_db}.{klines_table} created for {symbol} on {landing_date}")


INFO:__main__:Table crypto_cloud_dev_650251698703_serving_db.klines created for ADAUSDT on 2025-09-27


In [5]:
serving_db = f"{data_prefix}_serving_db"
klines_table = "klines"
spark.sql(f"""
select * from {serving_db}.{klines_table} order by group_id
""").show()

+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+
|group_id|         group_date|       open_time|open_price|high_price|low_price|close_price|  volume|      close_time|landing_date| symbol|
+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+
| 1954368|2025-09-27 00:00:00|1758931200686229|    0.7918|    0.7918|   0.7897|     0.7901|465779.1|1758932095846561|  2025-09-27|ADAUSDT|
| 1954369|2025-09-27 00:15:00|1758932100583189|    0.7902|    0.7924|     0.79|     0.7911|163062.6|1758932997788221|  2025-09-27|ADAUSDT|
| 1954370|2025-09-27 00:30:00|1758933000808905|    0.7912|     0.792|   0.7905|     0.7913|202817.4|1758933895280530|  2025-09-27|ADAUSDT|
| 1954371|2025-09-27 00:45:00|1758933900464103|    0.7913|    0.7927|   0.7899|     0.7923|434212.2|1758934797111165|  2025-09-27|ADAUSDT|
| 1954372|2025-09-27 01:00:

In [6]:
serving_db = f"{data_prefix}_serving_db"
pattern_two_table = "pattern_two"
spark.sql(f"""
select * from {serving_db}.{pattern_two_table} where pattern is not null
""").show(20, truncate=False)

+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+------+------+---------+-----------------+
|group_id|group_date         |open_time       |open_price|high_price|low_price|close_price|volume  |close_time      |landing_date|symbol |ema7  |ema20 |trend    |pattern          |
+--------+-------------------+----------------+----------+----------+---------+-----------+--------+----------------+------------+-------+------+------+---------+-----------------+
|1954413 |2025-09-27 11:15:00|1758971700405060|0.7837    |0.7853    |0.7837   |0.785      |121594.4|1758972591323187|2025-09-27  |ADAUSDT|0.7835|0.784 |downtrend|bullish engulfing|
|1954445 |2025-09-27 19:15:00|1759000502967967|0.7803    |0.7823    |0.78     |0.7821     |95233.0 |1759001384883232|2025-09-27  |ADAUSDT|0.7808|0.7817|downtrend|bullish engulfing|
+--------+-------------------+----------------+----------+----------+---------+-----------+----